In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import itertools 
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from scipy import stats
import datajoint as dj
from collections import defaultdict
import pycircstat as pycs
from funconnect import morphology, connectomics, ta3, nda
from tqdm import tqdm
import scipy.stats as stats


Connecting celiib@10.28.0.34:3306


In [8]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20


pinky = dj.create_virtual_module('pinky', 'microns_pinky')
pinky_nda = dj.create_virtual_module('pinky_nda', 'microns_pinky_nda')
pinky_radtune = dj.create_virtual_module('pinky_radtune', 'microns_pinky_radtune')
pinky_spattune = dj.create_virtual_module('pinky_spattune', 'microns_pinky_spattune')
pinky_fc = dj.create_virtual_module('pinky_fc', 'microns_pinky_fc')


In [4]:
#pinky.SharedInput() #did not exist

In [6]:
from utils import find_pearson, find_cosine, find_binary_sim

schema = dj.schema('microns_pinky_fc', create_tables=True)

@schema
class SharedInput(dj.Computed):
    definition = """    
    -> pinky.Segment
    segment_b           : bigint   # segment id unique within each Segmentation
    ---
    n_syn_a     : int # number of synapses for cell A
    n_syn_b     : int # number of synapses for cell B
    n_syn_union : int # number of unique synapses for both
    n_syn_shared: int # number of shared synapses
    n_seg_a     : int # number of segments for cell A
    n_seg_b     : int # number of segments for cell B
    n_seg_union : int # number of unique segments for both
    n_seg_shared: int # number of shared segments
    """

    key_source = pinky.Segmentation & pinky.CurrentSegmentation

    def make(self, key):
        #gets all of the segments with a trace entry minus the segmentExclusdes (all the excitatory and orphans)
        targets = (pinky.Segment & pinky_nda.Trace & key) - pinky.SegmentExclude 
        #gets all of the axons and the synapses where these axons are the presyns
        axons = pinky.Neurite & 'neurite_type="axon"'
        synapse = (pinky.Synapse & axons.proj(presyn='segment_id') & key) - pinky.SynapseExclude
        
        #get all the combinations of postsyn targets without any repeats
        info = targets * targets.proj(segment_b='segment_id') & 'segment_id < segment_b'
        
        #gets all of the synapses where the "segment_id" is the postsyn
        A = (synapse & targets.proj(postsyn='segment_id')).proj(
            'presyn', syn1='synapse_id', segment_id='postsyn')
        
        #gets all of the synapses where the "segment_id" is the postsyn and name it segment_b
        #only difference between this table and the previous one is the names syn2 and segment_b
        B = (synapse * dj.U('presyn') & targets.proj(postsyn='segment_id')).proj(
            'presyn', syn2='synapse_id', segment_b='postsyn')
        """
        A * B & 'segment_id < segment_b': will do every combination of synapse from that presyn that is on a different segment
        Then for each segment pair
        n_syn_shared: number of different synapse combinations from a common presyn for those to pairs
        n_seg_shared: number of distinct common presyns

        """
        shared = dj.U('segment_id', 'segment_b').aggr(A * B & 'segment_id < segment_b',
                                                      n_syn_shared='count(*)',
                                                      n_seg_shared='count(DISTINCT presyn)')
        
        
        a = dj.U('segment_id').aggr(A, n_syn_a='count(*)', n_seg_a='count(DISTINCT presyn)')
        b = dj.U('segment_b').aggr(B, n_syn_b='count(*)', n_seg_b='count(DISTINCT presyn)')
        stats = (info * a * b * shared).proj('n_syn_a', 'n_syn_b', 'n_syn_shared',
                                             'n_seg_a', 'n_seg_b', 'n_seg_shared',
                                             n_syn_union='n_syn_a + n_syn_b - n_syn_shared',
                                             n_seg_union='n_seg_a + n_seg_b - n_seg_shared')
        
        self.insert([dict(key, **k) for k in tqdm(stats.fetch(as_dict=True))],
                    ignore_extra_fields=True)
        
        non_shared = ((info - stats) * a * b).proj('n_syn_a', 'n_syn_b',
                                                    'n_seg_a', 'n_seg_b',
                                                    n_syn_union='n_syn_a + n_syn_b',
                                                    n_seg_union='n_seg_a + n_seg_b')
        self.insert([dict(key, **k, n_syn_shared=0, n_seg_shared=0) for k in tqdm(non_shared.fetch(as_dict=True))],
                    ignore_extra_fields=True)


In [11]:
key = pinky.Segmentation & pinky.CurrentSegmentation.fetch1()

In [16]:
targets = (pinky.Segment & pinky_nda.Trace & key) - pinky.SegmentExclude 
targets

segmentation segmentation id,segment_id segment id unique within each Segmentation
3,648518346341351894
3,648518346341352081
3,648518346341353058
3,648518346341353574
3,648518346341353607
3,648518346341353788
3,648518346341354313
3,648518346341354496
3,648518346341355048
3,648518346341355198


In [29]:
(926*926 - 926)/2,len(info)

(428275.0, 428275)

In [13]:
axons = pinky.Neurite & 'neurite_type="axon"'
synapse = (pinky.Synapse & axons.proj(presyn='segment_id') & key) - pinky.SynapseExclude

In [17]:
#get all the combinations of postsyn targets without any repeats
info = targets * targets.proj(segment_b='segment_id') & 'segment_id < segment_b'

A = (synapse & targets.proj(postsyn='segment_id')).proj(
    'presyn', syn1='synapse_id', segment_id='postsyn')

A

segmentation segmentation id,syn1 synapse index within the segmentation,presyn segment id unique within each Segmentation,segment_id segment id unique within each Segmentation
3,87581,648518346342926760,648518346341353058
3,110052,648518346341411143,648518346341353058
3,216848,648518346342802670,648518346341353058
3,244551,648518346349383182,648518346341353058
3,263412,648518346342920102,648518346341353058
3,270958,648518346349370927,648518346341353058
3,303029,648518346349372857,648518346341353058
3,321802,648518346349377832,648518346341353058
3,357482,648518346349387625,648518346341353058
3,400228,648518346341372480,648518346341353058


In [18]:
#gets all of the synapses where the "segment_id" is the postsyn but says that segment_b is the postsyn
B = (synapse * dj.U('presyn') & targets.proj(postsyn='segment_id')).proj(
    'presyn', syn2='synapse_id', segment_b='postsyn')

B

segmentation segmentation id,syn2 synapse index within the segmentation,presyn segment id unique within each Segmentation,segment_b segment id unique within each Segmentation
3,87581,648518346342926760,648518346341353058
3,110052,648518346341411143,648518346341353058
3,216848,648518346342802670,648518346341353058
3,244551,648518346349383182,648518346341353058
3,263412,648518346342920102,648518346341353058
3,270958,648518346349370927,648518346341353058
3,303029,648518346349372857,648518346341353058
3,321802,648518346349377832,648518346341353058
3,357482,648518346349387625,648518346341353058
3,400228,648518346341372480,648518346341353058


In [26]:
1744058/202500

8.612632098765433

In [25]:
A * B

segmentation segmentation id,syn1 synapse index within the segmentation,syn2 synapse index within the segmentation,presyn segment id unique within each Segmentation,segment_id segment id unique within each Segmentation,segment_b segment id unique within each Segmentation
3,87581,768367,648518346342926760,648518346341353058,648518346349471910
3,87581,180666,648518346342926760,648518346341353058,648518346349499773
3,87581,3116666,648518346342926760,648518346341353058,648518346349513012
3,87581,1072393,648518346342926760,648518346341353058,648518346349499369
3,87581,427790,648518346342926760,648518346341353058,648518346341398705
3,87581,87581,648518346342926760,648518346341353058,648518346341353058
3,87581,1851261,648518346342926760,648518346341353058,648518346349504324
3,87581,802903,648518346342926760,648518346341353058,648518346344072981
3,87581,140786,648518346342926760,648518346341353058,648518346349509855
3,87581,866798,648518346342926760,648518346341353058,648518346342921511


In [35]:
"""
A * B & 'segment_id < segment_b': will do every combination of synapse from that presyn that is on a different segment
Then for each segment pair
n_syn_shared: number of different synapse combinations from a common presyn for those to pairs
n_seg_shared: number of distinct common presyns

"""
shared = dj.U('segment_id', 'segment_b').aggr(A * B & 'segment_id < segment_b',
                                                      n_syn_shared='count(*)',
                                                      n_seg_shared='count(DISTINCT presyn)')

In [33]:
A

segmentation segmentation id,syn1 synapse index within the segmentation,presyn segment id unique within each Segmentation,segment_id segment id unique within each Segmentation
3,87581,648518346342926760,648518346341353058
3,110052,648518346341411143,648518346341353058
3,216848,648518346342802670,648518346341353058
3,244551,648518346349383182,648518346341353058
3,263412,648518346342920102,648518346341353058
3,270958,648518346349370927,648518346341353058
3,303029,648518346349372857,648518346341353058
3,321802,648518346349377832,648518346341353058
3,357482,648518346349387625,648518346341353058
3,400228,648518346341372480,648518346341353058


In [30]:
#number of synapse from one presyn and distinct presyn axons that attach to it
a = dj.U('segment_id').aggr(A, n_syn_a='count(*)', n_seg_a='count(DISTINCT presyn)') 
#number of synapse from one presyn and distinct presyn axons that attach to it (BUT JUST WITH DIFFERENT NAMES)
b = dj.U('segment_b').aggr(B, n_syn_b='count(*)', n_seg_b='count(DISTINCT presyn)')

In [31]:
a

segment_id segment id unique within each Segmentation,n_syn_a calculated attribute,n_seg_a calculated attribute
648518346341353058,119,103
648518346341353574,30,24
648518346341353607,103,90
648518346341354313,9,6
648518346341354496,70,55
648518346341355048,251,225
648518346341355198,28,25
648518346341355275,1,1
648518346341355736,237,224
648518346341356241,8,6


In [32]:
b

segment_b segment id unique within each Segmentation,n_syn_b calculated attribute,n_seg_b calculated attribute
648518346341353058,119,103
648518346341353574,30,24
648518346341353607,103,90
648518346341354313,9,6
648518346341354496,70,55
648518346341355048,251,225
648518346341355198,28,25
648518346341355275,1,1
648518346341355736,237,224
648518346341356241,8,6


In [36]:
info * a * b * shared

segmentation segmentation id,segment_id segment id unique within each Segmentation,segment_b segment id unique within each Segmentation,n_syn_a calculated attribute,n_seg_a calculated attribute,n_syn_b calculated attribute,n_seg_b calculated attribute,n_syn_shared calculated attribute,n_seg_shared calculated attribute
3,648518346341353058,648518346341355736,119,103,237,224,4,1
3,648518346341353058,648518346341356256,119,103,1029,920,3,3
3,648518346341353058,648518346341357933,119,103,88,72,1,1
3,648518346341353058,648518346341359123,119,103,290,271,7,6
3,648518346341353058,648518346341361981,119,103,760,690,3,3
3,648518346341353058,648518346341362995,119,103,659,589,3,3
3,648518346341353058,648518346341363175,119,103,381,331,1,1
3,648518346341353058,648518346341366161,119,103,105,94,1,1
3,648518346341353058,648518346341366757,119,103,238,213,12,10
3,648518346341353058,648518346341368711,119,103,144,128,1,1


In [ ]:
"""
1) starts with all info: all the combinations of postsyns without repeats
2) appends on the n_syn_a that 1st segment gets from all axons
3) appends on the n_syn_b that 2nd segment gets from all axons
4) For that combination of segments adds on:
    a. total number of different synapse combinations from a common presyn for those to pairs
    b. total number of common presyns
5) Then computes the following:
    a. n_syn_a + n_syn_b - (sum of all combinations of synapses from common presyns)
    --> so this could be negative hypothetically
    b. n_seg_a + n_seg_b - (total number of common presyns)
    --> aka the union of all presyns going to that pair


"""
stats = (info * a * b * shared).proj('n_syn_a', 'n_syn_b', 'n_syn_shared',
                                             'n_seg_a', 'n_seg_b', 'n_seg_shared',
                                             n_syn_union='n_syn_a + n_syn_b - n_syn_shared',
                                             n_seg_union='n_seg_a + n_seg_b - n_seg_shared')